# Solving the Hangman Game

## Lessons from the Baseline

The baseline `guess` function seems reasonable enough but achieves only about 18% accuracy, probably because it overlooks the fact that the test dictionary is completely different from the training one. It tries to always match input words _globally_ to the training dictionary, but the calculated letter frequencies will fail to reflect that of the input word as more matches are made, since no exact matches exist. 

This can be corroborated by using input words from the training set itself. My test shows that in this case the baseline `guess` will achieve ~90% accuracy.

## An Algorithm Based on Matching N-grams

Therefore, we will instead look for matching opportunities in local contexts. These are common letter combinations such as  _pro-_, _dis-_, _-tion_, and _-ing_. Word roots are good examples, but we'll just use n-grams to more generally represent these local contexts that are assumed to be equally common in the testing dictionary as in the training one.

The algorithm:

1. The n-gram probabilities (`ngram_ps`) are first calculated base on the training file during initialization. The default is to build probabilities for 2-, 3-, and 4-grams.

```
             0         1             2             3
h  3.778638e-07  0.000002  3.297863e-01  7.732188e-07
g  4.751117e-02  0.000002  1.453896e-01  3.240721e-02
d  4.524873e-03  0.000002  8.865222e-02  1.851840e-02
l  2.262437e-03  0.000002  3.900698e-02  1.111104e-01
u  2.262437e-03  0.000002  5.922546e-07  4.629601e-03
p  8.687757e-01  0.881169  1.028366e-01  7.732188e-07
v  3.778638e-07  0.000002  2.127653e-01  3.888865e-01
k  3.778638e-07  0.000002  3.546089e-03  7.732188e-07
w  2.262437e-02  0.000002  3.546089e-03  7.732188e-07
f  2.488680e-02  0.009901  1.418436e-02  8.333282e-02
b  2.714924e-02  0.108909  5.673742e-02  3.564793e-01
x  3.778638e-07  0.000002  3.546089e-03  7.732188e-07
y  3.778638e-07  0.000002  5.922546e-07  7.732188e-07
j  3.778638e-07  0.000002  5.922546e-07  7.732188e-07
z  3.778638e-07  0.000002  5.922546e-07  4.629601e-03
q  3.778638e-07  0.000002  5.922546e-07  7.732188e-07
```

2. During the guess, we maintain a probability matrix `p` (a pandas `Dataframe`) like the one shown above, whose rows are the letters and columns the positions in the input word. `p[a,j]` is the probability that letter `a` will be at position `j`. 

    - `p` will be initialized with the overall single letter probabilities (1-gram).

    - Look for a blank in the input word next to a run of k revealed positions. Together they corespond to a list of (k+1)-grams by allowing the blank to variate. Use the n-gram probabilities to update the corresponding columns of `p`. Word beginning and endings will be represented by `<` and `>` and also matched.

    - Any used letters and revealed positions are removed from the rows and columns of `p`. `p` is always normalized (all columns sum to 1).

3. When there are no more match opportunities, each remaining letter `a` in `p` will be assigned an overall probability of it appearing somewhere in the word.

```
p[a] = 1 - (1 - p[a, 0]) * (1 - p[a, 1]) * (1 - p[a, 2]) * ...
```
4. Finally, a letter is randomly selected, weighted by its overall probability.

## Implementation





In [1]:
import json
import requests
import pandas as pd
import numpy as np
import time
import collections

from urllib.parse import parse_qs, urlencode, urlparse

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
class HangmanTest(object):
    def __init__(self, access_token=None, session=None, timeout=None,
                max_ngram=5, # maximum 5-grams
                training_file="words_250000_train.txt"):
        
        with open(training_file) as file:
            self.full_dictionary = file.read().splitlines()
        self.current_dictionary = []
        
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout

        self.guessed_letters = set()

        self.max_ngram = max_ngram
        self.ngram_ps = dict()

        # init the n-gram probabilities 
        counts = collections.Counter("".join(self.full_dictionary)) # single letter
        self.ngram_ps[1] = pd.Series(counts) / counts.total()

        for n in range(2, self.max_ngram+1): # n-gram (n > 1)
            counts = collections.Counter(
                [f"<{w}>"[i:i+n] for w in self.full_dictionary for i in range(len(w) - n + 3)])
            self.ngram_ps[n] = pd.Series(counts) / counts.total()
        
        
    def gen_contextual_ps(self, dotted: str):
        """ generate the n-gram matching probabilities """
        if "." not in dotted:
            return
        
        chunks = dotted.split(".") # split to runs of known letters
        dot_index = -1   # index of the dot in input
        pattern = ""
        for j in range(len(chunks) - 1):
            combined = chunks[j] + "." + chunks[j+1]
            lj = len(chunks[j])
            dot_index = dotted.index(".", dot_index+1)

            if combined == ".": # no context around
                continue

            if len(combined) <= self.max_ngram:
                pattern = combined
            elif lj < self.max_ngram:
                pattern = combined[:self.max_ngram]
            else:
                pattern = combined[lj+1-self.max_ngram : lj+1]

            ps = self.ngram_ps[len(pattern)]
            i = pattern.index(".") # dot index in pattern
            ps = ps[ps.index.str.match(pattern.replace(".", "[a-z]"))].rename(
                index=lambda a: a[i]
            )
            yield dot_index-1, ps


    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, masked: str) -> str: # word input example: "_ p p _ e "
        # remove spaces, add start/end markers, and substitute . for _
        dotted = f"<{masked}>".translate(str.maketrans({"_": ".", " ": None}))
        
        p = pd.DataFrame(
            # default probabilities are those of single letters (1-gram)
            data={j: self.ngram_ps[1] for j in range(len(dotted)-2)},
            dtype=float)
        
        for column, ps in self.gen_contextual_ps(dotted):
            p.loc[:, column] = ps
        p.fillna(1e-10, inplace=True)

        # remove guessed letters (rows)
        p.drop(
            index=list(self.guessed_letters.intersection(p.index)),
            inplace=True)
        p = p.div(p.sum()) # normalize by column
        p.fillna(0., inplace=True)

        # remove revealed positions (columns)
        p.drop(
            columns=[j-1 for j, a in enumerate(dotted) if a.isalpha()], 
            inplace=True)

        # print("p = ", p, sep="\n")
                
        # guess the letter based on overall probability
        candidates = (1-(1-p).prod(axis="columns")).nlargest()
        guessed_letter = np.random.choice(
            a=candidates.index,
            p=candidates.values/candidates.sum())

        return guessed_letter


    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = set()
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.add(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

## Testing and Room for Improvement


Tests show that the algorithm implemented above can achieve ~35% accuracy. It is better than the baseline, but not as good as I expected. Some ways to improve the algorithm are:

1. Oftentimes, there are multiple alignments to match against an n-gram, some easier to solve than others. E.g., in the input word `comple_ion`, the 4-gram `?ion` is more likely to produce a correct result than `e?io`. However, in the current implementation no attempt is made to compare different alignments.

2. In the same game, the `p` matrix is recalculated from scratch for each guess. Measures should be taken to calculate _incrementally_.

In [3]:
api = HangmanTest(access_token="76e8540a498b33ed86bd8067eb0ba5", timeout=2000)

In [9]:
for i in range(100):
    print('Playing ', i+1, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=1, verbose=True)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    practice_success_rate = total_practice_successes / (total_practice_runs + 1e-10)
    success_rate = total_recorded_successes / (total_recorded_runs + 1e-10)
    print(f"- practice success rate: {practice_success_rate} (run {total_practice_runs} out of allotted 100,000).")
    print(f"- recorded success rate: {success_rate} (run {total_recorded_runs} out of allotted 1000).")

Playing  1  th game


HangmanAPIError: {'error': 'Your account has been deactivated!'}